Creating Volume to copy datafiles from github to Volumes 

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS workspace.default.store_source;


In [0]:
df = spark.read.format("csv")\
               .option("inferSchema", "true")\
                .option("multiLine", "true")\
                .option("quote",'"')\
                .option("escape",'"')\
                .option("header","true")\
                .load("/Volumes/workspace/default/store_source/superstore.csv")
df.display()

Adding bronze metadata column

In [0]:
from pyspark.sql.functions import current_timestamp, input_file_name

df_bronze = df.withColumn("ingestion_timestamp", current_timestamp())\
              .withColumn("file_source",df["_metadata.file_path"])
df_bronze.display()

In [0]:
df_bronze = df_bronze.withColumnRenamed("Product_ ID","Product_ID")\
                     .withColumnRenamed("Ship _Date","Ship_Date")
df_bronze.display()

In [0]:
df_bronze.printSchema()

Writing to bronze Layer under Data Engineering Schema

In [0]:
df_bronze.write.format("delta")\
         .mode("overwrite")\
          .option("overwriteSchema","true")\
         .saveAsTable("data_engineering_project.bronze_orders")

Checking the Load to bronze table 

In [0]:
df_bronze = spark.table("data_engineering_project.bronze_orders")

In [0]:
df_bronze.count()
df_bronze.printSchema()

In [0]:
from pyspark.sql.functions import col
df_bronze.where(col("Customer_Name") == 'Alex Avila').display()

In [0]:
%sql
INSERT INTO data_engineering_project.bronze_orders
VALUES (
  1161,
  'CA-2024-147039',
  '2024-07-10',              -- Order_Date (STRING)
  DATE '2024-07-15',
  'Standard Class',
  'AA-10315',
  'Alex Avila',
  'Corporate',               -- CHANGED
  'United States',
  'Chicago',                 -- CHANGED
  'Illinois',                -- CHANGED
  60616,
  'Central',
  'OFF-AP-10000576',
  'Office Supplies',
  'Appliances',
  'Belkin 325VA UPS Surge Protector, 6''',
  362.94,
  3,
  0.0,
  90.735,
  NULL,
  NULL,
  NULL,
  CURRENT_TIMESTAMP(),
  'scd2_test_customer'
);
